# Extracellular Electrophysiology Data

At the Allen Institute for Brain Science we carry out in vivo extracellular electrophysiology (ecephys) experiments  in awake animals using high-density Neuropixels probes. The data from these experiments are organized into *sessions*, where each session is a distinct continuous recording period. During a session we collect:

- spike times and characteristics (such as mean waveforms) from up to 6 neuropixels probes
- LFP data (TODO: not in yet!)
- behavioral data, such as running speed and eye position
- visual stimuli which were presented during the session
- cell-type specific optogenetic stimuli that were applied during the session

The AllenSDK contains code for accessing across-session (project-level) metadata as well as code for accessing detailed within-session data. The standard workflow is to use project-level tools, such as `EcephysProjectCache` to identify and access sessions of interest, then delve into those sessions' data using `EcephysSession`.


Project-level
------------------
The `EcephysProjectCache` class in `allensdk.brain_observatory.ecephys.ecephys_project_cache` accesses and stores data pertaining to many sessions. You can use this class to run queries that span all collected sessions and to download data for individual sessions.
* <a href='#Obtaining-an-EcephysProjectCache'>Obtaining an `EcephysProjectCache`</a>
* <a href='#Querying-sessions'>Querying sessions</a>
* <a href='#Querying-units'>Querying units</a>



Session-level
-------------------
The `EcephysSession` class in `allensdk.brain_observatory.ecephys.ecephys_session` provides an interface to all of the data for a single session, aligned to a common clock. This notebook will show you how to use the `EcephysSession` class to extract these data.
* <a href='#Obtaining-an-EcephysSession'>Obtaining an `EcephysSession`</a>
* <a href='#Stimulus-presentations'>Stimulus information</a>
* <a href='#Spike-data'>Spike data</a>
* <a href='#Spike-histograms'>Spike histograms</a>
* <a href='#Waveforms'>Unitwise mean waveforms</a>
* <a href='#Suggested-excercises'>Suggested excercises</a>

In [1]:
# first we need a bit of import boilerplate
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import EcephysSession, removed_unused_stimulus_presentation_columns
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

### Obtaining an `EcephysProjectCache`

In order to create an `EcephysProjectCache` object, you need to specify two things:
1. A remote source for the object to fetch data from. We will instantiate our cache using `EcephysProjectCache.from_warehouse()` to point the cache at the Allen Institute's public web API.
2. A path to a manifest json, which designates filesystem locations for downloaded data. The cache will try to read data from these locations before going to download those data from its remote source, preventing repeated downloads.

In [2]:
manifest_path = os.path.join('example_ecephys_project_cache', 'manifest.json')

cache = EcephysProjectCache.from_lims(manifest=manifest_path) # TODO: warehouse

### Querying across sessions

Using your `EcephysProjectCache`, you can download a table listing metadata for all sessions.

In [3]:
cache.get_sessions().head()

,name,stimulus_name,storage_directory,workflow_state,specimen_id,isi_experiment_id,equipment_id,operator_id,qc_operator_id,project_id,eye_tracking_id,observatory_stimulus_config_id,date_of_acquisition,created_at,updated_at,temp_upload_directory,published_at,habituation
id,,,,,,,,,,,,,,,,,,
828758609,20190225115927_678711844,NaN,/allen/programs/braintv/production/neuralcodin...,uploaded,807417255,815050020,677098837.0,678711844,NaN,714914585,NaN,NaN,2019-02-26 13:38:10.896908,2019-02-25 19:59:27.959054,2019-04-12 02:35:29.785036,//allen/programs/braintv/production/incoming/n...,NaN,False
779839471,20181115122458_647737834,NaN,/allen/programs/braintv/production/neuralcodin...,uploaded,760960653,766594733,677098837.0,647737834,NaN,714914585,NaN,NaN,2018-12-10 22:45:43.057893,2018-11-15 20:24:17.195195,2019-04-12 02:35:32.655418,//allen/programs/braintv/production/incoming/n...,NaN,False
778998620,20181114122508_704511589,NaN,/allen/programs/braintv/production/neuralcodin...,uploaded,759674770,763718472,677098837.0,704511589,NaN,714914585,NaN,NaN,2018-12-10 21:57:24.147149,2018-11-14 20:24:27.727474,2019-04-12 02:35:32.677501,//allen/programs/braintv/production/incoming/n...,NaN,False
759883607,20181002110403_678711844,NaN,/allen/programs/braintv/production/neuralcodin...,uploaded,740268986,745833491,677098837.0,678711844,NaN,714914585,NaN,NaN,2018-10-26 19:55:45.842645,2018-10-02 18:04:03.473049,2019-04-12 02:35:34.061659,//allen/programs/braintv/production/incoming/n...,NaN,False
759228117,20180927124208_647737834,NaN,/allen/programs/braintv/production/neuralcodin...,uploaded,734687127,742838007,677098837.0,647737834,NaN,714914585,NaN,NaN,2018-10-26 19:55:51.640158,2018-09-27 19:42:08.912752,2019-04-12 02:35:34.189767,//allen/programs/braintv/production/incoming/n...,NaN,False


### Querying across units

Likewise, you can obtain a table listing each sorted unit, as well as associated metadata.

In [4]:
units = cache.get_units()
units.head()

,local_index,cluster_ids,ecephys_channel_id,quality,snr,firing_rate,isi_violations
id,,,,,,,
849705364,0,0,849704604,noise,0.778123,3.243230,3.876614
849705366,1,1,849704602,good,1.446133,3.782304,0.784492
849705368,2,2,849704604,good,0.853575,5.839871,0.695103
849705370,3,3,849704620,noise,0.855205,6.976821,1.765077
849705372,4,4,849704620,noise,0.599069,4.632780,2.870669


In [ ]:
# There are quite a few of these
print(units.shape[0])

121669


### Obtaining an `EcephysSession`

We package each session's data into a Neurodata Without Borders 2.0 (NWB) file. Calling `get_session_data` on your `EcephysProjectCache` will download such a file and return an `EcephysSession` object.

`EcephysSession` objects contain methods and properties that access the data within an ecephys NWB file and cache it in memory.

In [ ]:
session_id = 797828357 # for example
session = cache.get_session_data(session_id)

/home/nile/Desktop/allensdk/allensdk/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_lims_api.py:26: UserWarning: copying a 750.963mb file from /allen/programs/braintv/production/neuralcoding/prod56/specimen_776061251/ecephys_session_797828357/EcephysWriteNwbStrategy/analysis_run_851887914/ecephys_session_797828357.nwb
  warnings.warn(f'copying a {fsize:.6}mb file from {main_nwb_path}')


We'll now jump in to accessing our session's data. If you ever want a complete documented list of the attributes and methods defined on `EcephysSession`, you can run `help(EcephysSession)` (or in a jupyter notebook: `EcephysSession?`).

### Sorted units

Units are putative neurons, clustered from raw voltage traces using Kilosort 2. Each unit is associated with a single *peak channel* on a single probe, though its spikes might be picked up with some attenuation on multiple nearby channels. Each unit is assigned a unique integer identifier ("unit_id") which can be used to look up its  spike times and its mean waveform.

The units for a session are recorded in an attribute called, fittingly, `units`. This is a `pandas.DataFrame` whose index is the unit id and whose columns contain summary information about the unit, its peak channel, and its associated probe.

In [ ]:
session.units.head()

As a `pandas.DataFrame` the units table supports many straightforward filtering operations:

In [ ]:
# how many units have signal to noise ratios that are greater than 4?
print(f'{session.units.shape[0]} units total')
units_with_very_high_snr = session.units[session.units['snr'] > 4]
print(f'{units_with_very_high_snr.shape[0]} units have snr > 4')

... as well as some more advanced (and very useful!) operations. For more information, please see the pandas documentation. The following topics might be particularly handy:

- [selecting data](http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)
- [merging multiple dataframes](http://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)
- [grouping rows within a dataframe](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)
- [pivot tables](http://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)

### Stimulus presentations

During the course of a session, visual stimuli are presented on a monitor to the subject. We call intervals of time where a specific stimulus is presented (and its parameters held constant!) a *stimulus presentation*.

You can find information about the stimulus presentations that were displayed during a session by accessing the `stimulus_presentations` attribute on your `EcephysSession` object. 

In [ ]:
session.stimulus_presentations.head()

Like the units table, this is a `pandas.DataFrame`. Each row corresponds to a stimulus presentation and lists the time (on the session's master clock, in seconds) when that presentation began and ended as well as the kind of stimulus that was presented (the "stimulus_name" column) and the parameter values that were used for that presentation. Many of these parameter values don't overlap between stimulus classes, so the stimulus_presentations table uses the string `"null"` to indicate an inapplicable parameter. The index is named "stimulus_presentation_id" and many methods on `EcephysSession` use these ids.

Some of the columns bear a bit of explanation:
- stimulus_block : A block consists of multiple presentations of the same stimulus class presented with (probably) different parameter values. If during a session stimuli were presented in the following order:
    - drifting gratings
    - static gratings
    - drifting gratings
    then the blocks for that session would be [0, 1, 2]. The gray period stimulus (just a blank gray screen) never gets a block.
- duration : this is just stop_time - start_time, precalculated for convenience.

What kinds of stimuli were presented during this session? Pandas makes it easy to find out:

In [ ]:
session.stimulus_names # just the unique values from the 'stimulus_name' column

If you are only interested in a subset of stimuli, you can either filter using pandas or using the `get_presentations_for_stimulus` convience method:

In [ ]:
session.get_presentations_for_stimulus(['contrast_response']).head()

We might also want to know what the total set of available parameters is. The `get_stimulus_parameter_values` method provides a dictionary mapping stimulus parameters to the set of values that were applied to those parameters:

In [ ]:
for key, values in session.get_stimulus_parameter_values().items():
    print(f'{key}: {values}')

What if we want parameter values for just one kind of stimulus, "drifting_gratings" for instance? Let's check the help for `get_stimulus_parameter_values`:

In [ ]:
help(session.get_stimulus_parameter_values)

From this help message we can see that `get_stimulus_parameter_values` takes an optional filtering parameter - an array of stimulus presentation ids (the index values from `EcephysSession.stimulus_presentations`). If we first find all of the stimulus presentations where drifting gratings were presented and then supply their ids to `get_stimulus_parameter_values`, we can find out what parameter values were used for drifting gratings in this session:

In [ ]:
drifting_gratings_presentations = session.stimulus_presentations[
    session.stimulus_presentations['stimulus_name'] == 'drifting_gratings'
]

# ".index.values" takes a dataframe  of presentations and returns a numpy array of presentation ids
drifting_gratings_presentation_ids = drifting_gratings_presentations.index.values 

drifting_gratings_parameter_values = session.get_stimulus_parameter_values(
    stimulus_presentation_ids=drifting_gratings_presentation_ids
)

for key, values in drifting_gratings_parameter_values.items():
    print(f'{key}: {values}')

It is not necessarily true that all possible parameter value combinations were presented. We can enumerate the combinations that were presented by calling `get_stimulus_conditions`. As with `get_stimulus_parameter_values` we can filter on stimulus presentation ids.

In [ ]:
session.get_stimulus_conditions(
    stimulus_presentation_ids=drifting_gratings_presentation_ids
).head(10)

### Spike data

The `EcephysSession` object holds spike times (in seconds on the session master clock) for each unit. These are stored in a dictionary, which maps unit ids (the index values of the units table) to arrays of spike times.

In [ ]:
 # grab an arbitrary (though high-snr!) unit (we made units_with_high_snr above)
high_snr_unit_ids = units_with_very_high_snr.index.values
unit_id = high_snr_unit_ids[0]

print(f'{len(session.spike_times[unit_id])} spikes were detected for unit {unit_id}')
session.spike_times[unit_id]

You can also obtain spikes tagged with the stimulus presentation during which they occurred:

In [ ]:
# get spike times from the first block of drifting gratings presentations 
drifting_gratings_presentation_ids = session.stimulus_presentations.loc[
    (session.stimulus_presentations['stimulus_name'] == 'drifting_gratings')
].index.values

times = session.presentationwise_spike_times(
    stimulus_presentation_ids=drifting_gratings_presentation_ids,
    unit_ids=high_snr_unit_ids
)

times.head()

We can make raster plots of these data:

In [ ]:
first_drifting_grating_presentation_id = times['stimulus_presentation_id'].values[0]
plot_times = times[times['stimulus_presentation_id'] == first_drifting_grating_presentation_id]

fig = raster_plot(plot_times, title=f'spike raster for stimulus presentation {first_drifting_grating_presentation_id}')
plt.show()

# also print out this presentation
session.stimulus_presentations.loc[first_drifting_grating_presentation_id]

What if we want to know how many spikes a given stimulus condition evoked? We can call `conditionwise_spike_counts` on our session object

In [ ]:
contrast_response_presentation_ids = session.stimulus_presentations[
    session.stimulus_presentations['stimulus_name'].isin(set(['contrast_response']))
].index.values


contrast_response_spike_counts = session.conditionwise_spike_counts(
    unit_ids=high_snr_unit_ids, 
    stimulus_presentation_ids=contrast_response_presentation_ids
)

# filter out inapplicable stimulus columns for cleaner display
contrast_response_spike_counts = removed_unused_stimulus_presentation_columns(contrast_response_spike_counts)

contrast_response_spike_counts.head()

We can also get mean spike counts per stimulus condition:

In [ ]:
mean_spike_counts = session.conditionwise_mean_spike_counts(
    unit_ids=high_snr_unit_ids, 
    stimulus_presentation_ids=contrast_response_presentation_ids
)
mean_spike_counts = removed_unused_stimulus_presentation_columns(mean_spike_counts)
mean_spike_counts.head()

Using this mean spike counts dataframe, we can ask for each unit: which stimulus condition evoked the most activity on average?

In [ ]:
max_rate_conditions = mean_spike_counts.groupby('unit_id')\
    .apply(lambda df: df.loc[df['mean_spike_count'].idxmax()])
max_rate_conditions.drop(columns='unit_id', inplace=True)
max_rate_conditions.head()

### Spike histograms

It is commonly useful to compare spike data from across units and stimulus presentations, all relative to the onset of a stimulus presentation. We can do this using the `presentationwise_spike_counts` method. 

In [ ]:
# We're going to build an array of spike counts surrounding stimulus presentation onset
# To do that, we will need to specify some bins (in seconds, relative to stimulus onset)
time_bin_edges = np.linspace(-0.01, 0.4, 200)

# look at responses to the flash stimulus
flash_250_ms_stimulus_presentation_ids = session.stimulus_presentations[
    session.stimulus_presentations['stimulus_name'] == 'flash_250ms'
].index.values

# and get a set of units with only decent snr
decent_snr_unit_ids = session.units[
    session.units['snr'] >= 1.5
].index.values

spike_counts_ds = session.presentationwise_spike_counts(
    bin_edges=time_bin_edges,
    stimulus_presentation_ids=flash_250_ms_stimulus_presentation_ids,
    unit_ids=decent_snr_unit_ids
)
spike_counts_ds

This has returned a new (to this notebook) data structure, the `xarray.DataSet`. You can think of this as similar to a 3+D `pandas.DataFrame`, or as a `numpy.ndarray` with labeled axes and indices. See the [xarray documentation](http://xarray.pydata.org/en/stable/index.html) for more information. In the mean time, the salient features are:

- Data variables : One or more (in our case one) multidimensional arrays of data, similar to `numpy.ndarray`s. We can access our data array of spike times like: `ds['spike_counts']`.
- Dimensions : Each axis on each data variable is associated with a named dimension. This lets us see unambiguously what the axes of our array mean.
- Coordinates : Arrays of labels for each sample on each dimension.

xarray is nice because it forces code to be explicit about dimensions and coordinates, improving readability and avoiding bugs. However, you can always convert to numpy or pandas data structures as follows:
- to pandas: `spike_counts_ds['spike_counts'].to_dataframe()` produces a multiindexed dataframe
- to numpy: `spike_counts_ds['spike_counts'].values` gives you access to the underlying numpy array

We can now plot spike counts for a particular presentation:

In [ ]:
presentation_id = 3796 # chosen arbitrarily
plot_spike_counts(
    spike_counts_ds['spike_counts'].loc[{'stimulus_presentation_id': presentation_id}], 
    spike_counts_ds['time_relative_to_stimulus_onset'],
    'spike count', 
    f'unitwise spike counts on presentation {presentation_id}'
)
plt.show()

We can also average across all presentations, adding a new data array to the dataset. Notice that this one no longer has a stimulus_presentation_id dimension, as we have collapsed it by averaging.

In [ ]:
spike_counts_ds['mean_spike_counts'] = spike_counts_ds['spike_counts'].mean(dim='stimulus_presentation_id')
spike_counts_ds

... and plot the mean spike counts

In [ ]:
plot_spike_counts(
    spike_counts_ds['mean_spike_counts'], 
    spike_counts_ds['time_relative_to_stimulus_onset'],
    'mean spike count', 
    'mean spike counts on flash_250_ms presentations'
)
plt.show()

### Waveforms

We store precomputed mean waveforms for each unit in the `mean_waveforms` attribute on the `EcephysSession` object. This is a dictionary which maps unit ids to xarray dataarrays. These have channel and time (seconds, aligned to the detected event times) dimensions. The data values are millivolts, as measured at the recording site.

In [ ]:
first_fifteen_units_waveforms = {uid: session.mean_waveforms[uid] for uid in high_snr_unit_ids[:15]}
first_fifteen_units_peak_channels = {uid: session.units.loc[uid, 'peak_channel_id'] for uid in high_snr_unit_ids[:15]}

# plot the mean waveform on each unit's peak channel/
plot_mean_waveforms(first_fifteen_units_waveforms, high_snr_unit_ids[:15], first_fifteen_units_peak_channels)
plt.show()

### Running Speed

We can obtain the velocity at which the experimental subject ran as a function of time by accessing the `running_speed` attribute. This returns a [named tuple](https://docs.python.org/3/library/typing.html?#typing.NamedTuple) with two fields: timestamps and values. Timestamps report when on the session's master clock samples where taken, while values report the linear velocity (cm/s) of the subject at that time. Negative values indicate backwards running.

Here we'll plot the running speed trace for a single arbitrary stimulus presentation:

In [ ]:
presentation_3807_rs_timestamp_indices = np.searchsorted(
    session.running_speed.timestamps,
    [
        session.stimulus_presentations.loc[3807, 'start_time'], 
        session.stimulus_presentations.loc[3807, 'stop_time']
    ]
)

plot_running_speed(
    session.running_speed.timestamps, 
    session.running_speed.values, 
    start_index=presentation_3807_rs_timestamp_indices[0],
    stop_index=presentation_3807_rs_timestamp_indices[1]
)
plt.show()

### Suggested excercises

If you would hands-on experience with the `EcephysSession` class, please consider working through some of these excercises.

- **tuning curves** : Pick a stimulus parameter, such as orientation on drifting gratings trials. Plot the mean and standard error of spike counts for each unit at each value of this parameter.
- **signal correlations** : Calculate unit-pairwise correlation coefficients on the tuning curves for a stimulus parameter of interest (`numpy.corrcoef` might be useful).
- **noise correlations** : Build for each unit a vector of spike counts across repeats of the same stimulus condition. Compute unit-unit correlation coefficients on these vectors.
- **cross-correlations** : Start with two spike trains. Call one of them "fixed" and the other "moving". Choose a set of time offsets and for each offset:
    1. apply the offset to the spike times in the moving train
    2. compute the correlation coefficient between the newly offset moving train and the fixed train.
    You should then be able to plot the obtained correlation coeffients as a function of the offset. 
- **unit clustering** : First, extract a set of unitwise features. You might draw these from the mean waveforms, for instance:
    - mean duration between waveform peak and trough (on the unit's peak channel)
    - the amplitude of the unit's trough
    
    or you might draw them from the unit's spike times, such as:
    - median inter-spike-interval
    
    or from metadata
    - CCF structure
    
    With your features in hand, attempt an unsupervised classification of the units. If this seems daunting, check out the [scikit-learn unsupervised learning documention](https://scikit-learn.org/stable/modules/clustering.html#clustering) for library code and examples.
- **population decoding** : Using an `EcephysSession` (and filtering to some stimuli and units of interest), build two aligned matrices:
    1. A matrix whose rows are stimulus presentations, columns are units, and values are spike counts.
    2. A matrix whose rows are stimulus presentations and whose columns are stimulus parameters.
    
    Using these matrices, train a classifier to predict stimulus conditions (sets of stimulus parameter values) from presentationwise population spike counts. See the [scikit-learn supervised learning tutorial](https://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html) for a guide to supervised learning in Python.